<a href="https://colab.research.google.com/github/friedelj/AAI-510-TEAM-03/blob/main/JFriedel_CV_Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment 3 Exercise: Object Detection and Segmentation
Part 1- Object Segmentation
Object detection is a task in computer vision that involves identifying the presence, location, and type of one or more objects in a given image. In order to have an accurate object detection model, we need to train a model based of thousands or millions of images related to that object and this makes the task even more challenging. As was mentioned in week 2, even after finding appropriate data, training the model is not an easy task. We discussed one solution which was using pre-trained models. COCO is a large-scale object detection, segmentation, and captioning dataset.
a) Load the image dataset oxford_iiit from tf
b) Create the segmentation mask on the first 3 samples in the dataset. Show both the image and the segmented version of it. Remember to map the images into 128x128.[link text](https://)

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pylab as plt
import numpy as np
import cv2

In [ ]:
# #a) Load the oxford_iiit_pet dataset
# Load the Oxford-IIIT Pet dataset
dataset, info = tfds.load('oxford_iiit_pet', split='train', with_info=True)

# Function to preprocess the image and segmentation mask
def preprocess_sample(sample):
    # Resize images and masks to 128x128
    image = tf.image.resize(sample['image'], [128, 128])
    mask = tf.image.resize(sample['segmentation_mask'], [128, 128])

    # Normalize the image to [0, 1]
    image = tf.cast(image, tf.float32) / 255.0

    return image, mask

# Preprocess the dataset
dataset = dataset.map(preprocess_sample)

# Display function to show images and segmentation masks
def display_images_and_masks(dataset, num_samples=3):
    plt.figure(figsize=(10, num_samples * 5))
    for i, (image, mask) in enumerate(dataset.take(num_samples)):
        # Show the image
        plt.subplot(num_samples, 2, 2*i + 1)

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

In [3]:
# Resize, Normalize, Adjust Mask
def read_and_preprocess(data):
    # Resize the image and segmentation mask to 128x128
    input_image = tf.image.resize(data['image'], [128, 128])
    input_mask = tf.image.resize(data['segmentation_mask'], [128, 128])

    # Normalize the image to [0, 1]
    input_image = tf.image.convert_image_dtype(input_image, tf.float32)

    # Adjust the mask values from {1,2,3} to {0,1,2}
    input_mask = tf.cast(input_mask, tf.int32) - 1

    return input_image, input_mask

In [4]:
# Load the Oxford-IIIT Pet dataset
dataset, info = tfds.load('oxford_iiit_pet', with_info=True)

# Map the read_and_preprocess function onto train and test datasets
train = dataset['train'].map(read_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test = dataset['test'].map(read_and_preprocess, num_parallel_calls=tf.data.AUTOTUNE)

# Optional: Batch, shuffle, and prefetch the train and test datasets for efficiency
train = train.cache().shuffle(1000).batch(32).prefetch(tf.data.AUTOTUNE)
test = test.batch(32).prefetch(tf.data.AUTOTUNE)

# Display information
print(f"Number of samples in the train set: {info.splits['train'].num_examples}")
print(f"Number of samples in the test set: {info.splits['test'].num_examples}")

NotImplementedError: `.as_dataset()` not implemented for ArrayRecord files. Please, use `.as_data_source()`.

In [5]:
# # b) Create the segmentation mask
# Show some images from dataset and their segmented version
# Function to display images and their segmentation masks
def display_images_and_masks(dataset, num_samples=3):
    plt.figure(figsize=(10, num_samples * 5))
    for i, (image, mask) in enumerate(dataset.take(num_samples)):
        # Display the original image
        plt.subplot(num_samples, 2, 2 * i + 1)
        plt.imshow(image)
        plt.title("Image")
        plt.axis("off")

        # Display the segmentation mask
        plt.subplot(num_samples, 2, 2 * i + 2)
        plt.imshow(mask[:, :, 0], cmap='gray')  # Show the mask in grayscale
        plt.title("Segmentation Mask")
        plt.axis("off")
    plt.show()

# Show images and masks from the train dataset
display_images_and_masks(train)

NameError: name 'train' is not defined

Assignment 3 Exercise: Object Detection and Segmentation
Part 1- Object Segmentation
Object detection is a task in computer vision that involves identifying the presence, location, and type of one or more objects in a given image. In order to have an accurate object detection model, we need to train a model based of thousands or millions of images related to that object and this makes the task even more challenging. As was mentioned in week 2, even after finding appropriate data, training the model is not an easy task. We discussed one solution which was using pre-trained models. COCO is a large-scale object detection, segmentation, and captioning dataset.
a) Load the image dataset oxford_iiit from tf
b) Create the segmentation mask on the first 3 samples in the dataset. Show both the image and the segmented version of it. Remember to map the images into 128x128.
b) A class of fubctions is provided for you that creates mask and images. Use the provided class and create a sample of image and its mask.

Part 3 YOLO
In 2016, researchers at Washington University, Allen Institute for AI, and Facebook AI Research proposed “You Only Look Once” (YOLO), a family of neural networks that improved the speed and accuracy of object detection with deep learning.
The main improvement in YOLO is the integration of the entire object detection and classification process in a single network. Instead of extracting features and regions separately, YOLO performs everything in a single pass through a single network, hence the name “You Only Look Once.”
YOLOv10, developed by researchers at Tsinghua University using the Ultralytics Python package, offers a novel approach to real-time object detection. It addresses post-processing and model architecture shortcomings found in previous YOLO versions. By removing non-maximum suppression (NMS) and optimizing various model components, YOLOv10 achieves state-of-the-art performance with significantly reduced computational overhead. Extensive experiments highlight its superior accuracy-latency trade-offs across multiple model scales. You can read more about it here.
a)  For the first part of the assignment, we want to see how to create the annotation for a custom dataset. You can use the provided html file (annotation_tool.html). When you open the file, upload an image of your desire and assign the proper class. Here you see an image of a raccoon and its chosen
class.

In [1]:
import os
HOME = os.getcwd()
!pip install -q git+https://github.com/THU-MIG/yolov10.git
!mkdir -p {HOME}/weights
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10n.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10s.pt

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [3]:
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10m.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10b.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10x.pt
!wget -P {HOME}/weights -q https://github.com/THU-MIG/yolov10/releases/download/v1.1/yolov10l.pt

c)  Now, we want to test the model out of box. There is an image stored in: “/content/yolov10/ultralytics/assets/bus.jpg”. Apply YOLO10 command to make predictions.
After making the prediction, the result will be stored in “/content/runs/detect/predict/bus.jpg”. Call the image and show the result of prediction.
d)  Now, is the time to try to fine-tune the model. Train the Yolo10n model on coco128.ml. For this part use 10 epochs and 32 batches. Show the confusion matrix which is stored in:
{HOME}/runs/detect/train/confusion_matrix.png
Next, show the learning curves which are stored in: {HOME}/runs/detect/train/results.png
e)  Explain what you see in the learning curves and confusion matrix.
